## Data Scrape
### In this file, I use excel file that has hyperlink along with rows, access all website using beautifulsoup to get the information 
    Steps
         1- Read excel file using openpyxl library to get the hyperlinks.  
         2- Use request library and beautifulsoup to scrape the data using tags e.g., <section>, <div>, <p>.
         3- Iter through existed dataframe to fill empty rows. 

In [253]:
import pandas as pd
from openpyxl import load_workbook
import requests
from bs4 import BeautifulSoup
import time
import numpy as np

In [298]:
# Extracting data from Excel sheet
wb=load_workbook('KSA COMP ESTABLISHED  DATE.xlsx')
ws=wb['Sheet1']
data=[]
for row in ws.iter_rows():
    row_data=[]
    for cell in row:
        if cell.hyperlink:
            cell_value=cell.hyperlink.target
        else:
            cell_value=cell.value
        row_data.append(cell_value)
    data.append(row_data)


In [348]:
# function using beautifulSoup to get the data from websites. 
def scarp_establishment_date(link):
    try:
        proxies = {
        'http': '127.0.0.1 :50401',
#         'https': '127.0.0.1 :50401',
        }
        response=requests.get(link,timeout=10,proxies=proxies)
        soup=BeautifulSoup(response.content,'html.parser')
        # code to get company's name 
        name_part=soup.find('section',{'class':'saudiPage saudi-cable width80'})
        company_name=name_part.find('h3').text.strip()

        # code to get establishment date.
        date_part=soup.find('div',{'class':'company_management_tab_dtl'})
        establish_date=date_part.find_all('p')[0].text
#         df=pd.DataFrame(columns=('company_name','establish_date'))
    except Exception as e:
        print("An error occurred:", e)
    return company_name,establish_date

In [349]:
# Creating DataFrame and cleaning data
linkdf=pd.DataFrame(data)
linkdf.columns=['Company_link','Estab_date']
linkdf=linkdf.drop(linkdf.index[0],axis=0)
linkdf=linkdf.iloc[lambda x: x.index%2!=0]
linkdf['Company_name']=np.nan
linkdf.reset_index(drop=True,inplace=True)

linkdf.head()

,Company_link,Estab_date,Company_name
0,https://www.saudiexchange.sa/wps/portal/saudie...,None,NaN
1,https://www.saudiexchange.sa/wps/portal/saudie...,None,NaN
2,https://www.saudiexchange.sa/wps/portal/saudie...,None,NaN
3,https://www.saudiexchange.sa/wps/portal/saudie...,None,NaN
4,https://www.saudiexchange.sa/wps/portal/saudie...,None,NaN


In [370]:
# call a function to fill the dataframe, Do iteration 
for index, row in linkdf.iterrows():
    linkdf.loc[index,['Estab_date','Company_name']]=scarp_establishment_date(row[0])   
    

In [373]:
# export dataframe to excel file
linkdf.to_excel('establish_date.xlsx')

In [374]:
linkdf.head()

,Company_link,Estab_date,Company_name
0,https://www.saudiexchange.sa/wps/portal/saudie...,Saudi Arabia Refineries Co.,1960/06/14
1,https://www.saudiexchange.sa/wps/portal/saudie...,Saudi Arabian Oil Co.,1988/11/13
2,https://www.saudiexchange.sa/wps/portal/saudie...,Rabigh Refining and Petrochemical Co.,2005/09/19
3,https://www.saudiexchange.sa/wps/portal/saudie...,Arabian Drilling Co.,1964/04/07
4,https://www.saudiexchange.sa/wps/portal/saudie...,National Shipping Company of Saudi Arabia,1978/01/22
